In [ ]:
from os.path import join, abspath, dirname, isfile
from os import listdir

import sys
import json
sys.path.insert(0, abspath(join(dirname("."), '..')))

import pickle as pkl
import pandas as pd

from random import seed
from numpy import genfromtxt

from modules.helpers.utils import path_decompose
from modules.Simulator import *


In [ ]:
DS_PATH = path_decompose("H:\\Research\\NIGraphs\\withPasive")
SEED_PATH = path_decompose("H:\\Research\\NIGraphs\\seeds")
path_to_json = '../results/'

In [ ]:
simmulations_to_run = 10
threads_to_use = 22
CURRENT_SEED = 2024

In [ ]:
seed(CURRENT_SEED)

In [ ]:
all_graphs = [f for f in listdir(DS_PATH) if isfile(join(DS_PATH, f))]
all_seeds = [f for f in listdir(SEED_PATH) if isfile(join(SEED_PATH, f))]

In [ ]:
gaph_data = {}

for graph_name in all_graphs:
    graph_config = graph_name.split("fromData")[0].split("]")
    num_nodes = int(graph_config[0].replace("[", ""))
    num_edges= int(graph_config[1].replace("[", ""))

    # Read Graph Data
    graph_path = join(DS_PATH, graph_name)

    gaph_data[f"{num_nodes}_{num_edges}"] = pkl.load(open(graph_path, 'rb'))

In [ ]:
seed_data = {}

for seed_name in all_seeds:
        seed_tokens = seed_name.split("fromData")
        graph_config = seed_tokens[0].split("]")
        num_nodes = int(graph_config[0].replace("[", ""))
        num_edges= int(graph_config[1].replace("[", ""))
        percentile = seed_tokens[1].split("_")[1]

        # Read Graph Data
        seed_nodes = genfromtxt(join(SEED_PATH, seed_name), delimiter=",")

        seed_data[f"{num_nodes}_{num_edges}_{percentile}"] = seed_nodes

In [ ]:
configurations = {}
blocked_nodes = {}
sim_config = pd.read_csv(join(path_to_json, 'simulationData.csv'))
sim_blocked = pd.read_csv(join(path_to_json, 'simulationNodes.csv'))

for index, row in sim_config.iterrows():
    num_nodes = row["num_nodes"]
    num_edges = row["num_edges"]
    percentile = row["percentile_name"]
    id = row["id"]

    composed_key = (f"{num_nodes}_{num_edges}_{percentile}", f"{num_nodes}_{num_edges}")

    if not composed_key in configurations:
        configurations[composed_key] = []

    configurations[composed_key].append(id)    

for index, row in sim_blocked.iterrows():
    blocked_nodes[row['id']] = [int(x) for x in row['nodes'][1:-2].split(",")]


In [ ]:
sim_results = []
for composed_key, ids in configurations.items():
    (seed_key, graph_key) = composed_key
    G = gaph_data[graph_key]
    graph_seed = seed_data[seed_key]

    sim = Simulator(G, graph_seed)

    for id in ids:
        sim.add_blocked(id, blocked_nodes[id])
        results = sim.run(simmulations_to_run, threads_to_use)

        sim_results.append(results)

In [ ]:

with open(join(path_to_json, "simulations", "sim_results.json"), 'w') as f:
    json.dump(sim_results, f)